<h1 style='color:red' align='center'> Health Insurance Lead Prediction </h4>

<h3 style='color:blue'> Importing the libraries </h3>

In [315]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import MinMaxScaler,LabelEncoder,StandardScaler
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
le = LabelEncoder()
pca=PCA()
sc=StandardScaler()

import tensorflow as tf
from tensorflow import keras

import warnings
warnings.filterwarnings('ignore')

df_train = pd.read_csv('train_Df64byy.csv')
df_test = pd.read_csv('test_YCcRUnU.csv')
df_train.set_index('ID',inplace=True)
df_test.set_index('ID',inplace=True)

<h3 style = 'color : blue'> Exploratory Data Analysis </h3>

In [316]:
df_train.head()

,ID,City_Code,Region_Code,Accomodation_Type,Reco_Insurance_Type,Upper_Age,Lower_Age,Is_Spouse,Health Indicator,Holding_Policy_Duration,Holding_Policy_Type,Reco_Policy_Cat,Reco_Policy_Premium,Response
0,1,C3,3213,Rented,Individual,36,36,No,X1,14+,3.0,22,11628.0,0
1,2,C5,1117,Owned,Joint,75,22,No,X2,NaN,NaN,22,30510.0,0
2,3,C5,3732,Owned,Individual,32,32,No,NaN,1.0,1.0,19,7450.0,1
3,4,C24,4378,Owned,Joint,52,48,No,X1,14+,3.0,19,17780.0,0
4,5,C8,2190,Rented,Individual,44,44,No,X2,3.0,1.0,16,10404.0,0


In [317]:
df_train.columns

Index(['ID', 'City_Code', 'Region_Code', 'Accomodation_Type',
       'Reco_Insurance_Type', 'Upper_Age', 'Lower_Age', 'Is_Spouse',
       'Health Indicator', 'Holding_Policy_Duration', 'Holding_Policy_Type',
       'Reco_Policy_Cat', 'Reco_Policy_Premium', 'Response'],
      dtype='object')

In [318]:
df_train.dtypes

ID                           int64
City_Code                   object
Region_Code                  int64
Accomodation_Type           object
Reco_Insurance_Type         object
Upper_Age                    int64
Lower_Age                    int64
Is_Spouse                   object
Health Indicator            object
Holding_Policy_Duration     object
Holding_Policy_Type        float64
Reco_Policy_Cat              int64
Reco_Policy_Premium        float64
Response                     int64
dtype: object

<h3 style='color:blue'> Altering the data for the best fit </h3>

In [319]:
# Replacing the data as 0 and 1

df_train['Accomodation_Type'].replace({'Rented':0, 'Owned':1},inplace=True)
df_train['Reco_Insurance_Type'].replace({'Individual':0, 'Joint':1}, inplace=True)
df_train['Is_Spouse'].replace({'No':0, 'Yes':1}, inplace=True)

# Considered the 14+ as 15
df_train['Holding_Policy_Duration'].replace({'14+' : 15}, inplace=True)
df_train['Holding_Policy_Duration'].replace({'15' : 15}, inplace=True)

# Converted the new value 15 into numeric value
df_train['Holding_Policy_Duration'] = pd.to_numeric(df_train['Holding_Policy_Duration'],errors='coerce')

df_train['Health Indicator']=df_train['Health Indicator'].fillna(method="ffill")
df_train['Holding_Policy_Type']=df_train['Holding_Policy_Type'].fillna(method="ffill")
df_train['Holding_Policy_Duration']=df_train['Holding_Policy_Duration'].fillna(method="ffill")

# Dropped the below columns
df_train.drop(['ID','Region_Code'],axis=1,inplace=True)

# Label encoded the below column
df_train['Health Indicator']=le.fit_transform(df_train['Health Indicator'])
df_train['City_Code']=le.fit_transform(df_train['City_Code'])

# Converted the age between 0 to 4 accoring to the limit
df_train.loc[df_train['Upper_Age'] <= 20, 'Upper_Age'] = 0
df_train.loc[(df_train['Upper_Age'] > 20) & (df_train['Upper_Age'] <= 60), 'Upper_Age'] = 1
df_train.loc[(df_train['Upper_Age'] > 60) & (df_train['Upper_Age'] <= 80), 'Upper_Age'] = 2
df_train.loc[ df_train['Upper_Age'] > 80, 'Upper_Age'] = 3

df_train.loc[df_train['Lower_Age'] <= 20, 'Lower_Age'] = 0
df_train.loc[(df_train['Lower_Age'] > 20) & (df_train['Lower_Age'] <= 60), 'Lower_Age'] = 1
df_train.loc[(df_train['Lower_Age'] > 60) & (df_train['Lower_Age'] <= 80), 'Lower_Age'] = 2
df_train.loc[ df_train['Lower_Age'] > 80, 'Lower_Age'] = 3

# Tried MinMaxScaler to convert the below column values between 0 to 1
mms_cols = ['Reco_Policy_Cat','Reco_Policy_Premium','City_Code','Upper_Age','Lower_Age']
mms=MinMaxScaler()
df_train[mms_cols] = mms.fit_transform(df_train[mms_cols])

<h3 style = 'color : blue'> Applying the above method in test data set also </h3>

In [320]:
# Replacing the data as 0 and 1

df_test['Accomodation_Type'].replace({'Rented':0, 'Owned':1},inplace=True)
df_test['Reco_Insurance_Type'].replace({'Individual':0, 'Joint':1}, inplace=True)
df_test['Is_Spouse'].replace({'No':0, 'Yes':1}, inplace=True)

# Considered the 14+ as 15
df_test['Holding_Policy_Duration'].replace({'14+' : 15}, inplace=True)
df_test['Holding_Policy_Duration'].replace({'15' : 15}, inplace=True)

# Converted the new value 15 into numeric value
df_test['Holding_Policy_Duration'] = pd.to_numeric(df_test['Holding_Policy_Duration'],errors='coerce')

df_test['Health Indicator']=df_test['Health Indicator'].fillna(method="ffill")
df_test['Holding_Policy_Type']=df_test['Holding_Policy_Type'].fillna(method="ffill")
df_test['Holding_Policy_Duration']=df_test['Holding_Policy_Duration'].fillna(method="ffill")

# Dropped the below columns
df_test.drop(['ID','Region_Code'],axis=1,inplace=True)

# Label encoded the below column
df_test['Health Indicator']=le.fit_transform(df_test['Health Indicator'])
df_test['City_Code']=le.fit_transform(df_test['City_Code'])

# Converted the age between 0 to 3 accoring to the limit
df_test.loc[df_test['Upper_Age'] <= 20, 'Upper_Age'] = 0
df_test.loc[(df_test['Upper_Age'] > 20) & (df_test['Upper_Age'] <= 60), 'Upper_Age'] = 1
df_test.loc[(df_test['Upper_Age'] > 60) & (df_test['Upper_Age'] <= 80), 'Upper_Age'] = 2
df_test.loc[ df_test['Upper_Age'] > 80, 'Upper_Age'] = 3

df_test.loc[df_test['Lower_Age'] <= 20, 'Lower_Age'] = 0
df_test.loc[(df_test['Lower_Age'] > 20) & (df_test['Lower_Age'] <= 60), 'Lower_Age'] = 1
df_test.loc[(df_test['Lower_Age'] > 60) & (df_test['Lower_Age'] <= 80), 'Lower_Age'] = 2
df_test.loc[ df_test['Lower_Age'] > 80, 'Lower_Age'] = 3

# Tried MinMaxScaler to convert the below column values between 0 to 1
mms_cols = ['Reco_Policy_Cat','Reco_Policy_Premium','City_Code']
mms=MinMaxScaler()
df_test[mms_cols] = mms.fit_transform(df_test[mms_cols])



In [321]:
df_train.shape

(50882, 12)

<h3 style = 'color : blue'> Checking datatypes and unique values before splitting the value </h3>

In [322]:
df_train.dtypes

City_Code                  float64
Accomodation_Type            int64
Reco_Insurance_Type          int64
Upper_Age                  float64
Lower_Age                  float64
Is_Spouse                    int64
Health Indicator             int32
Holding_Policy_Duration    float64
Holding_Policy_Type        float64
Reco_Policy_Cat            float64
Reco_Policy_Premium        float64
Response                     int64
dtype: object

In [323]:
for i in df_train:
    print(f'{i} : {df_train[i].unique()}')

City_Code : [0.62857143 0.88571429 0.45714286 0.97142857 1.         0.
 0.17142857 0.57142857 0.54285714 0.94285714 0.34285714 0.48571429
 0.85714286 0.31428571 0.77142857 0.02857143 0.22857143 0.25714286
 0.2        0.6        0.74285714 0.51428571 0.28571429 0.91428571
 0.08571429 0.11428571 0.05714286 0.14285714 0.4        0.42857143
 0.37142857 0.82857143 0.71428571 0.65714286 0.8        0.68571429]
Accomodation_Type : [0 1]
Reco_Insurance_Type : [0 1]
Upper_Age : [0.5 1.  0. ]
Lower_Age : [0.5 1.  0. ]
Is_Spouse : [0 1]
Health Indicator : [0 1 3 2 5 4 7 6 8]
Holding_Policy_Duration : [15.  1.  3.  5.  9. 14.  7.  2. 11. 10.  8.  6.  4. 13. 12.]
Holding_Policy_Type : [3. 1. 4. 2.]
Reco_Policy_Cat : [1.         0.85714286 0.71428571 0.76190476 0.         0.80952381
 0.95238095 0.57142857 0.9047619  0.38095238 0.04761905 0.14285714
 0.52380952 0.23809524 0.61904762 0.47619048 0.0952381  0.33333333
 0.28571429 0.42857143 0.66666667 0.19047619]
Reco_Policy_Premium : [0.22760918 0.68735

<h3 style='color : blue'> Splitting the dataframe into xtrain, ytrain </h3>

In [324]:
xtrain=df_train.drop('Response',axis=1)
ytrain=df_train['Response']

In [325]:
ytrain.value_counts()

0    38673
1    12209
Name: Response, dtype: int64

<h3 style='color : blue'> Appling LDA to reduce the dimensionality </h3>

In [326]:
xtrain.shape

(50882, 11)

In [327]:
xtest=df_test

lda = LDA()
xtrain = lda.fit_transform(xtrain,ytrain)
xtest = lda.transform(xtest)

xtrain = sc.fit_transform(xtrain,ytrain)
xtest = sc.transform(xtest)

In [328]:
xtrain.shape

(50882, 1)

<h3 style='color : blue'> Imbalanced dataset, so regularaizing it </h3>

In [329]:
from imblearn.over_sampling import SMOTE
smote = SMOTE(sampling_strategy='minority')
x_train, y_train = smote.fit_sample(xtrain,ytrain)

y_train.value_counts()

1    38673
0    38673
Name: Response, dtype: int64

<h3 style='color : blue'> Built an Artificial Neural Network to train the dataset </h3>

In [330]:
ann = keras.Sequential([
    keras.layers.Dense(1, input_shape=(1,),activation='relu'),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(8,activation='relu'),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(1,activation='sigmoid')
])

ann.compile(optimizer='adam',
           loss='binary_crossentropy',
           metrics=['accuracy'])

ann.fit(x_train, y_train,epochs=100)

Epoch 1/100
2418/2418 [==============================] - 3s 1ms/step - loss: 0.6857 - accuracy: 0.5392
Epoch 2/100
2418/2418 [==============================] - 3s 1ms/step - loss: 0.6843 - accuracy: 0.5446
Epoch 3/100
2418/2418 [==============================] - 3s 1ms/step - loss: 0.6837 - accuracy: 0.5435
Epoch 4/100
2418/2418 [==============================] - 3s 1ms/step - loss: 0.6826 - accuracy: 0.5473
Epoch 5/100
2418/2418 [==============================] - 3s 1ms/step - loss: 0.6836 - accuracy: 0.5428
Epoch 6/100
2418/2418 [==============================] - 3s 1ms/step - loss: 0.6828 - accuracy: 0.5448
Epoch 7/100
2418/2418 [==============================] - 3s 1ms/step - loss: 0.6832 - accuracy: 0.5407
Epoch 8/100
2418/2418 [==============================] - 3s 1ms/step - loss: 0.6832 - accuracy: 0.5420
Epoch 9/100
2418/2418 [==============================] - 3s 1ms/step - loss: 0.6837 - accuracy: 0.5432
Epoch 10/100
2418/2418 [==============================] - 3s 1ms/step - l

<h3 style='color : blue'> Predicting the output for the test dataset </h3>

In [331]:
y_pred_nn = ann.predict(xtest)
y_pred_nn = np.round(y_pred_nn).astype(int)

In [332]:
y_pred_nn = y_pred_nn.flatten()

<h3 style='color : blue'> Creating a new dataframe for the submission </h3>

In [333]:
testData = pd.read_csv('test_YCcRUnU.csv')
ANN_Submission = pd.DataFrame({
        "ID": testData['ID'],
        "Response": y_pred_nn
    })

ANN_Submission['Response'].value_counts()

1    17199
0     4606
Name: Response, dtype: int64

In [334]:
ANN_Submission.to_csv('Health_Insurance_Lead_Prediction.csv',index=False)

<h3 style='color : Red'> Thank you!!! </h3>

*** End of the file